In [2]:
import cv2
import numpy as np
import mediapipe as mp
from scipy.spatial import distance as dist
import time
import pygame
import threading
from collections import deque
import math

pygame 2.6.1 (SDL 2.28.4, Python 3.10.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [23]:
# class MediaPipeDrowsinessDetector:
#     def __init__(self):
#         # Initialize MediaPipe Face Mesh
#         self.mp_face_mesh = mp.solutions.face_mesh
#         self.mp_drawing = mp.solutions.drawing_utils
#         self.mp_drawing_styles = mp.solutions.drawing_styles
        
#         # Initialize Face Mesh model
#         self.face_mesh = self.mp_face_mesh.FaceMesh(
#             max_num_faces=1,
#             refine_landmarks=True,
#             min_detection_confidence=0.5,
#             min_tracking_confidence=0.5
#         )
        
#         # COMPREHENSIVE MediaPipe Face Mesh landmark indices with MAXIMUM POINTS around eyes and mouth
#         # Left eye landmarks - ALL possible eye-related points for maximum precision
#         self.LEFT_EYE = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246, 
#                         130, 25, 110, 24, 23, 22, 26, 112, 243, 190, 56, 28, 27, 29, 30, 247, 31, 226, 
#                         113, 225, 224, 223, 222, 221, 189, 244, 245, 122, 6, 202, 214, 234, 227, 116, 
#                         117, 118, 119, 120, 121, 128, 126, 142, 36, 205, 206, 207, 213, 192, 147, 123, 
#                         124, 139, 71, 68, 104, 69, 108, 151, 35, 31, 228, 229, 230, 231, 232, 233, 244, 
#                         245, 122, 6, 202, 214, 234, 227, 116, 117, 118, 119, 120, 121, 128, 126, 142, 
#                         36, 205, 206, 207, 213, 192, 147, 123, 124, 139, 71, 68, 104, 69, 108, 9, 10, 
#                         151, 337, 299, 333, 298, 301, 251, 284, 332, 297, 338, 10, 109, 67, 103, 54, 21]
        
#         # Right eye landmarks - ALL possible eye-related points for maximum precision
#         self.RIGHT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398,
#                          359, 255, 339, 254, 253, 252, 256, 341, 463, 414, 286, 258, 257, 259, 260, 467,
#                          261, 446, 342, 445, 444, 443, 442, 441, 413, 464, 435, 410, 454, 365, 397, 288,
#                          361, 340, 346, 347, 348, 349, 350, 451, 452, 453, 464, 435, 410, 454, 365, 397,
#                          288, 361, 340, 346, 347, 348, 349, 350, 451, 452, 453, 464, 435, 410, 278, 294,
#                          295, 296, 334, 293, 300, 276, 283, 282, 295, 285, 336, 285, 336, 296, 334, 293,
#                          300, 276, 283, 282, 448, 449, 450, 451, 452, 453, 464, 435, 410, 454, 365, 397,
#                          288, 361, 340, 346, 347, 348, 349, 350, 451, 463, 414, 286, 258, 257, 259, 260]
        
#         # ENHANCED: More precise eye landmarks for EAR calculation
#         # Left eye: outer corner, multiple top points, inner corner, multiple bottom points
#         self.LEFT_EYE_SIMPLE = [33, 160, 159, 158, 133, 153, 145, 144]  # 8 points for better precision
#         # Right eye: outer corner, multiple top points, inner corner, multiple bottom points  
#         self.RIGHT_EYE_SIMPLE = [362, 385, 386, 387, 263, 373, 374, 380]  # 8 points for better precision
        
#         # COMPREHENSIVE: Maximum mouth landmarks for superior MAR calculation
#         self.MOUTH_SIMPLE = [0, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 24, 25, 26, 27, 28, 29, 30, 31, 32, 
#                             33, 37, 39, 40, 41, 42, 61, 62, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 84, 85, 
#                             86, 87, 88, 89, 90, 91, 95, 96, 146, 178, 179, 180, 181, 183, 184, 185, 186, 191, 
#                             267, 268, 269, 270, 271, 272, 273, 291, 292, 302, 303, 304, 305, 306, 307, 308, 
#                             310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 324, 325, 375, 402, 
#                             403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 415, 308, 324, 318, 402, 317, 
#                             14, 87, 178, 88, 95, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 324, 318, 
#                             405, 320, 307, 375, 321, 308, 324, 318, 267, 269, 270, 267, 271, 272]
        
#         # Head pose estimation landmarks
#         self.FACE_OVAL = [10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103, 67]
        
#         # ADJUSTED: More realistic thresholds
#         self.EAR_THRESHOLD = 0.21
#         self.MAR_THRESHOLD = 0.7   
#         self.HEAD_POSE_THRESHOLD = 45  # FIXED: Increased threshold to reduce false positives
#         self.CONSECUTIVE_FRAMES = 25   
        
#         # Counters and tracking variables
#         self.ear_counter = 0
#         self.mar_counter = 0
#         self.head_pose_counter = 0
#         self.blink_counter = 0
#         self.total_blinks = 0
        
#         # Temporal tracking
#         self.ear_history = deque(maxlen=30)
#         self.mar_history = deque(maxlen=30)
#         self.head_pose_history = deque(maxlen=30)
#         self.blink_history = deque(maxlen=300)  # 10 seconds at 30fps
        
#         # FIXED: Head pose calibration - store initial pose as reference
#         self.reference_pose = None
#         self.pose_calibrated = False
#         self.calibration_frames = 0
#         self.calibration_poses = []
        
#         # Alert system with continuous beeping
#         pygame.mixer.init()
#         self.alert_sound = None
#         self.load_alert_sound()
#         self.alert_playing = False
#         self.alert_channel = None
        
#         # Previous eye state for blink detection
#         self.prev_eye_closed = False
        
#         # Time tracking
#         self.start_time = time.time()
        
#     def load_alert_sound(self):
#         """Load alert sound with continuous beeping capability"""
#         try:
#             # Try to load a custom alert sound
#             self.alert_sound = pygame.mixer.Sound('alert.wav')
#         except:
#             # Create a continuous beep sound programmatically
#             sample_rate = 22050
#             duration = 0.5  # Shorter beep for continuous effect
#             frequency = 1000  # Higher frequency for urgency
#             t = np.linspace(0, duration, int(sample_rate * duration))
            
#             # Create a more urgent beep pattern
#             wave = np.sin(frequency * 2 * np.pi * t) * 0.4
#             # Add fade in/out to avoid clicking
#             fade_samples = int(0.05 * sample_rate)  # 50ms fade
#             wave[:fade_samples] *= np.linspace(0, 1, fade_samples)
#             wave[-fade_samples:] *= np.linspace(1, 0, fade_samples)
            
#             sound_array = (wave * 32767).astype(np.int16)
#             sound_array = np.repeat(sound_array.reshape(len(sound_array), 1), 2, axis=1)
#             self.alert_sound = pygame.sndarray.make_sound(sound_array)
    
#     def get_landmarks(self, landmarks, indices, width, height):
#         """Extract specific landmarks and convert to pixel coordinates"""
#         points = []
#         for idx in indices:
#             x = int(landmarks.landmark[idx].x * width)
#             y = int(landmarks.landmark[idx].y * height)
#             points.append([x, y])
#         return np.array(points)
    
#     def eye_aspect_ratio(self, eye_landmarks):
#         """Enhanced Eye Aspect Ratio (EAR) calculation using multiple points"""
#         try:
#             # Multiple vertical distances for better accuracy
#             A1 = dist.euclidean(eye_landmarks[1], eye_landmarks[7])  # Top to bottom (vertical 1)
#             A2 = dist.euclidean(eye_landmarks[2], eye_landmarks[6])  # Top to bottom (vertical 2)
#             A3 = dist.euclidean(eye_landmarks[1], eye_landmarks[6])  # Cross vertical
#             A4 = dist.euclidean(eye_landmarks[2], eye_landmarks[7])  # Cross vertical
            
#             # Horizontal distance (width of the eye)
#             C = dist.euclidean(eye_landmarks[0], eye_landmarks[4])  # Outer corner to inner corner
            
#             # Enhanced EAR formula using multiple measurements
#             if C == 0:  # Prevent division by zero
#                 return 0.3
            
#             ear = (A1 + A2 + A3 + A4) / (4.0 * C)
#             return ear
#         except:
#             return 0.3
    
#     def mouth_aspect_ratio(self, mouth_landmarks):
#         """Enhanced Mouth Aspect Ratio (MAR) calculation using multiple points"""
#         try:
#             # Multiple vertical mouth measurements for better accuracy
#             A1 = dist.euclidean(mouth_landmarks[1], mouth_landmarks[7])   # Top to bottom (center)
#             A2 = dist.euclidean(mouth_landmarks[2], mouth_landmarks[6])   # Top to bottom (left-center)
#             A3 = dist.euclidean(mouth_landmarks[3], mouth_landmarks[5])   # Top to bottom (right-center)
#             A4 = dist.euclidean(mouth_landmarks[13], mouth_landmarks[19]) # Inner vertical distance
#             A5 = dist.euclidean(mouth_landmarks[14], mouth_landmarks[18]) # Inner vertical distance
            
#             # Horizontal mouth distance (width)
#             C1 = dist.euclidean(mouth_landmarks[0], mouth_landmarks[4])   # Left corner to right corner
#             C2 = dist.euclidean(mouth_landmarks[12], mouth_landmarks[16]) # Inner horizontal distance
            
#             if C1 == 0 or C2 == 0:  # Prevent division by zero
#                 return 0.3
                
#             # Enhanced MAR formula using multiple measurements
#             mar = (A1 + A2 + A3 + A4 + A5) / (3.0 * (C1 + C2))
#             return mar
#         except:
#             return 0.3
    
#     def get_head_pose_mediapipe(self, landmarks, width, height):
#         """Estimate head pose using MediaPipe 3D landmarks"""
#         # Key 3D model points for head pose estimation
#         model_points = np.array([
#             (0.0, 0.0, 0.0),             # Nose tip
#             (0.0, -330.0, -65.0),        # Chin
#             (-225.0, 170.0, -135.0),     # Left eye left corner
#             (225.0, 170.0, -135.0),      # Right eye right corner  
#             (-150.0, -150.0, -125.0),    # Left mouth corner
#             (150.0, -150.0, -125.0)      # Right mouth corner
#         ])
        
#         # Corresponding 2D image points from MediaPipe landmarks
#         image_points = np.array([
#             [landmarks.landmark[1].x * width, landmarks.landmark[1].y * height],    # Nose tip
#             [landmarks.landmark[152].x * width, landmarks.landmark[152].y * height], # Chin
#             [landmarks.landmark[33].x * width, landmarks.landmark[33].y * height],   # Left eye corner
#             [landmarks.landmark[263].x * width, landmarks.landmark[263].y * height], # Right eye corner
#             [landmarks.landmark[61].x * width, landmarks.landmark[61].y * height],   # Left mouth corner
#             [landmarks.landmark[291].x * width, landmarks.landmark[291].y * height]  # Right mouth corner
#         ], dtype="double")
        
#         # Camera internals
#         focal_length = width
#         center = (width/2, height/2)
#         camera_matrix = np.array([
#             [focal_length, 0, center[0]],
#             [0, focal_length, center[1]],
#             [0, 0, 1]
#         ], dtype="double")
        
#         dist_coeffs = np.zeros((4,1))
        
#         # Solve PnP
#         try:
#             success, rotation_vector, translation_vector = cv2.solvePnP(
#                 model_points, image_points, camera_matrix, dist_coeffs, 
#                 flags=cv2.SOLVEPNP_ITERATIVE
#             )
            
#             if success:
#                 # Convert rotation vector to angles
#                 rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
#                 angles, _, _, _, _, _ = cv2.RQDecomp3x3(rotation_matrix)
#                 return angles[0], angles[1], angles[2]  # pitch, yaw, roll
#             else:
#                 return 0, 0, 0
#         except:
#             return 0, 0, 0
    
#     def calibrate_head_pose(self, head_angles):
#         """FIXED: Calibrate reference head pose for better detection"""
#         if not self.pose_calibrated:
#             self.calibration_poses.append(head_angles)
#             self.calibration_frames += 1
            
#             if self.calibration_frames >= 30:  # Calibrate over 30 frames
#                 # Calculate average pose as reference
#                 avg_pitch = np.mean([pose[0] for pose in self.calibration_poses])
#                 avg_yaw = np.mean([pose[1] for pose in self.calibration_poses])
#                 avg_roll = np.mean([pose[2] for pose in self.calibration_poses])
                
#                 self.reference_pose = (avg_pitch, avg_yaw, avg_roll)
#                 self.pose_calibrated = True
#                 print(f"Head pose calibrated: P:{avg_pitch:.1f} Y:{avg_yaw:.1f} R:{avg_roll:.1f}")
                
#         return self.pose_calibrated
    
#     def detect_blink(self, ear):
#         """Detect blinks based on EAR"""
#         eye_closed = ear < self.EAR_THRESHOLD
        
#         # Detect transition from closed to open (blink completion)
#         if self.prev_eye_closed and not eye_closed:
#             self.total_blinks += 1
#             self.blink_history.append(time.time())
        
#         self.prev_eye_closed = eye_closed
#         return eye_closed
    
#     def get_blink_frequency(self):
#         """Calculate blinks per minute"""
#         current_time = time.time()
#         # Remove blinks older than 60 seconds
#         self.blink_history = deque([t for t in self.blink_history if current_time - t <= 60], 
#                                   maxlen=300)
#         return len(self.blink_history)
    
#     def analyze_drowsiness(self, ear, mar, head_angles, blink_freq):
#         """Multi-factor drowsiness analysis"""
#         drowsiness_indicators = {
#             'eye_closure': False,
#             'yawning': False,
#             'head_nodding': False,
#             'low_blink_rate': False
#         }
        
#         drowsiness_score = 0
        
#         # FIXED: Eye closure detection with better logic
#         if ear < self.EAR_THRESHOLD:
#             self.ear_counter += 1
#             if self.ear_counter >= self.CONSECUTIVE_FRAMES:
#                 drowsiness_indicators['eye_closure'] = True
#                 drowsiness_score += 40
#         else:
#             self.ear_counter = 0
        
#         # FIXED: Yawn detection with better threshold
#         if mar > self.MAR_THRESHOLD:
#             self.mar_counter += 1
#             if self.mar_counter >= 15:  # Increased threshold for yawning
#                 drowsiness_indicators['yawning'] = True
#                 drowsiness_score += 25
#         else:
#             self.mar_counter = 0
        
#         # FIXED: Head pose analysis with calibration
#         if self.pose_calibrated:
#             pitch, yaw, roll = head_angles
#             ref_pitch, ref_yaw, ref_roll = self.reference_pose
            
#             # Calculate deviations from reference pose
#             pitch_deviation = abs(pitch - ref_pitch)
#             yaw_deviation = abs(yaw - ref_yaw)
#             roll_deviation = abs(roll - ref_roll)
            
#             # Only trigger if significant deviation from normal pose
#             if (pitch_deviation > self.HEAD_POSE_THRESHOLD or 
#                 yaw_deviation > self.HEAD_POSE_THRESHOLD or 
#                 roll_deviation > self.HEAD_POSE_THRESHOLD):
#                 self.head_pose_counter += 1
#                 if self.head_pose_counter >= 30:  # Increased threshold
#                     drowsiness_indicators['head_nodding'] = True
#                     drowsiness_score += 20
#             else:
#                 self.head_pose_counter = 0
        
#         # Low blink rate detection (normal: 15-20 blinks/minute)
#         if blink_freq < 6:  # Very low blink rate
#             drowsiness_indicators['low_blink_rate'] = True
#             drowsiness_score += 15
        
#         # Store history for temporal analysis
#         self.ear_history.append(ear)
#         self.mar_history.append(mar)
#         self.head_pose_history.append(head_angles)
        
#         return drowsiness_indicators, drowsiness_score
    
#     def trigger_alert(self, drowsiness_score, indicators):
#         """Trigger appropriate alerts with continuous beeping for critical states"""
#         if drowsiness_score >= 60:  # High drowsiness
#             alert_level = "CRITICAL"
#             color = (0, 0, 255)  # Red
#             should_beep = True
#         elif drowsiness_score >= 35:  # Moderate drowsiness
#             alert_level = "WARNING"
#             color = (0, 165, 255)  # Orange
#             should_beep = True
#         elif drowsiness_score >= 20:  # Mild drowsiness
#             alert_level = "CAUTION"
#             color = (0, 255, 255)  # Yellow
#             should_beep = False
#         else:
#             alert_level = "NORMAL"
#             color = (0, 255, 0)  # Green
#             should_beep = False
        
#         # CONTINUOUS BEEPING LOGIC
#         if should_beep and self.alert_sound:
#             if not self.alert_playing:
#                 # Start continuous beeping
#                 try:
#                     self.alert_channel = self.alert_sound.play(-1)  # -1 means loop indefinitely
#                     self.alert_playing = True
#                     print(f"🚨 ALERT STARTED: {alert_level}")
#                 except:
#                     pass
#         else:
#             # Stop beeping if alert level is low
#             if self.alert_playing:
#                 try:
#                     if self.alert_channel:
#                         self.alert_channel.stop()
#                     self.alert_playing = False
#                     print(f"✅ ALERT STOPPED: {alert_level}")
#                 except:
#                     pass
        
#         return alert_level, color
    
#     def run_detection(self):
#         """Main detection loop using MediaPipe"""
#         cap = cv2.VideoCapture(0)
        
#         # Set camera properties
#         cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
#         cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
#         cap.set(cv2.CAP_PROP_FPS, 30)
        
#         print("MediaPipe Drowsiness Detection System Started...")
#         print("Press 'q' to quit")
#         print(f"EAR Threshold: {self.EAR_THRESHOLD}")
#         print(f"MAR Threshold: {self.MAR_THRESHOLD}")
#         print(f"Consecutive Frames: {self.CONSECUTIVE_FRAMES}")
#         print("Calibrating head pose... Please look straight ahead for a few seconds")
        
#         while True:
#             ret, frame = cap.read()
#             if not ret:
#                 break
            
#             # Flip frame horizontally for mirror effect
#             frame = cv2.flip(frame, 1)
#             height, width, _ = frame.shape
            
#             # Convert BGR to RGB for MediaPipe
#             rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#             rgb_frame.flags.writeable = False
            
#             # Process the frame with MediaPipe
#             results = self.face_mesh.process(rgb_frame)
            
#             # Convert back to BGR
#             rgb_frame.flags.writeable = True
            
#             if results.multi_face_landmarks:
#                 for face_landmarks in results.multi_face_landmarks:
#                     # Extract eye landmarks for EAR calculation
#                     left_eye = self.get_landmarks(face_landmarks, self.LEFT_EYE_SIMPLE, width, height)
#                     right_eye = self.get_landmarks(face_landmarks, self.RIGHT_EYE_SIMPLE, width, height)
                    
#                     # Extract mouth landmarks for MAR calculation
#                     mouth = self.get_landmarks(face_landmarks, self.MOUTH_SIMPLE, width, height)
                    
#                     # Calculate ratios
#                     left_ear = self.eye_aspect_ratio(left_eye)
#                     right_ear = self.eye_aspect_ratio(right_eye)
#                     ear = (left_ear + right_ear) / 2.0
                    
#                     mar = self.mouth_aspect_ratio(mouth)
                    
#                     # Get head pose
#                     head_angles = self.get_head_pose_mediapipe(face_landmarks, width, height)
                    
#                     # FIXED: Calibrate head pose first
#                     self.calibrate_head_pose(head_angles)
                    
#                     # Detect blinks and calculate frequency
#                     self.detect_blink(ear)
#                     blink_freq = self.get_blink_frequency()
                    
#                     # Analyze drowsiness
#                     indicators, drowsiness_score = self.analyze_drowsiness(
#                         ear, mar, head_angles, blink_freq
#                     )
                    
#                     # Trigger alerts
#                     alert_level, alert_color = self.trigger_alert(drowsiness_score, indicators)
                    
#                     # UPDATED: Draw eye and mouth landmarks WITHOUT contour lines - only points
#                     left_eye_full = self.get_landmarks(face_landmarks, self.LEFT_EYE, width, height)
#                     right_eye_full = self.get_landmarks(face_landmarks, self.RIGHT_EYE, width, height)
#                     mouth_full = self.get_landmarks(face_landmarks, self.MOUTH_SIMPLE, width, height)
                    
#                     # REMOVED: All contour line drawing (drawContours calls)
                    
#                     # Draw individual landmark points with different colors and sizes
#                     # Left eye points in cyan
#                     for point in left_eye_full:
#                         cv2.circle(frame, tuple(point), 2, (255, 255, 0), -1)  # Cyan, size 2
                    
#                     # Right eye points in cyan
#                     for point in right_eye_full:
#                         cv2.circle(frame, tuple(point), 2, (255, 255, 0), -1)  # Cyan, size 2
                    
#                     # Mouth points in yellow
#                     for point in mouth_full:
#                         cv2.circle(frame, tuple(point), 2, (0, 255, 255), -1)  # Yellow, size 2
                    
#                     # Draw face bounding rectangle
#                     face_points = self.get_landmarks(face_landmarks, self.FACE_OVAL, width, height)
#                     x, y, w, h = cv2.boundingRect(face_points)
#                     cv2.rectangle(frame, (x, y), (x + w, y + h), alert_color, 2)
                    
#                     # Display metrics
#                     y_offset = 30
#                     cv2.putText(frame, f"Alert Level: {alert_level}", (10, y_offset), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, alert_color, 2)
                    
#                     y_offset += 30
#                     cv2.putText(frame, f"Drowsiness Score: {drowsiness_score}", (10, y_offset), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                    
#                     y_offset += 25
#                     cv2.putText(frame, f"EAR: {ear:.3f} (Thresh: {self.EAR_THRESHOLD})", (10, y_offset), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                    
#                     y_offset += 20
#                     cv2.putText(frame, f"MAR: {mar:.3f} (Thresh: {self.MAR_THRESHOLD})", (10, y_offset), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                    
#                     y_offset += 20
#                     if self.pose_calibrated:
#                         # Show deviation from reference pose
#                         ref_pitch, ref_yaw, ref_roll = self.reference_pose
#                         pitch_dev = abs(head_angles[0] - ref_pitch)
#                         yaw_dev = abs(head_angles[1] - ref_yaw)
#                         roll_dev = abs(head_angles[2] - ref_roll)
#                         cv2.putText(frame, f"Head Deviation: P:{pitch_dev:.1f} Y:{yaw_dev:.1f} R:{roll_dev:.1f}", 
#                                    (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
#                     else:
#                         cv2.putText(frame, f"Calibrating... ({self.calibration_frames}/30)", 
#                                    (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
                    
#                     y_offset += 20
#                     cv2.putText(frame, f"Blinks/min: {blink_freq}", (10, y_offset), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                    
#                     y_offset += 20
#                     cv2.putText(frame, f"Total Blinks: {self.total_blinks}", (10, y_offset), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                    
#                     y_offset += 20
#                     cv2.putText(frame, f"EAR Counter: {self.ear_counter}/{self.CONSECUTIVE_FRAMES}", (10, y_offset), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                    
#                     # Display active indicators
#                     y_offset += 25
#                     for indicator, active in indicators.items():
#                         if active:
#                             cv2.putText(frame, f"{indicator.replace('_', ' ').title()}: ACTIVE", 
#                                        (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
#                             y_offset += 20
            
#             # Show frame
#             cv2.imshow('MediaPipe Drowsiness Detection System', frame)
            
#             # Break loop on 'q' key press
#             if cv2.waitKey(1) & 0xFF == ord('q'):
#                 break
        
#         # Cleanup
#         cap.release()
#         cv2.destroyAllWindows()
        
#         # Stop any playing alerts
#         if self.alert_playing and self.alert_channel:
#             try:
#                 self.alert_channel.stop()
#             except:
#                 pass
                
#         self.face_mesh.close()
#         pygame.mixer.quit()
class MediaPipeDrowsinessDetector:
    def __init__(self):
        # Initialize MediaPipe Face Mesh
        self.mp_face_mesh = mp.solutions.face_mesh
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_drawing_styles = mp.solutions.drawing_styles
        
        # Initialize Face Mesh model
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        
        # ALL MediaPipe Face Mesh landmark indices - COMPLETE 468 POINTS
        # Left eye region - ALL points around left eye area (indices 0-230 approximately)
        self.LEFT_EYE = list(range(0, 17)) + list(range(18, 23)) + list(range(24, 35)) + \
                       list(range(36, 42)) + list(range(54, 68)) + list(range(69, 76)) + \
                       list(range(103, 134)) + list(range(139, 155)) + list(range(157, 173)) + \
                       list(range(189, 246)) + [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161]
        
        # Right eye region - ALL points around right eye area (indices 230-468 approximately)  
        self.RIGHT_EYE = list(range(249, 280)) + list(range(283, 300)) + list(range(334, 365)) + \
                        list(range(373, 390)) + list(range(398, 415)) + list(range(435, 468)) + \
                        [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384]
        
        # Mouth region - ALL points around mouth, lips, and surrounding area
        self.MOUTH_COMPREHENSIVE = list(range(0, 17)) + list(range(61, 96)) + list(range(146, 180)) + \
                                  list(range(267, 325)) + list(range(375, 415)) + \
                                  [78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308, 415, 310, 311, 312]
        
        # ENHANCED: More precise eye landmarks for EAR calculation
        # Left eye: outer corner, multiple top points, inner corner, multiple bottom points
        self.LEFT_EYE_SIMPLE = [33, 160, 159, 158, 133, 153, 145, 144]  # 8 points for better precision
        # Right eye: outer corner, multiple top points, inner corner, multiple bottom points  
        self.RIGHT_EYE_SIMPLE = [362, 385, 386, 387, 263, 373, 374, 380]  # 8 points for better precision
        
        # COMPREHENSIVE: Maximum mouth landmarks for superior MAR calculation
        self.MOUTH_SIMPLE = list(range(0, 17)) + list(range(61, 96)) + list(range(146, 180)) + \
                           list(range(267, 325)) + list(range(375, 415)) + \
                           [78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308, 415, 310, 311, 312]
        
        # OPTION: Use ALL 468 facial landmarks for complete coverage
        self.ALL_FACE_LANDMARKS = list(range(468))  # All 468 MediaPipe landmarks
        
        # Head pose estimation landmarks
        self.FACE_OVAL = [10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103, 67]
        
        # ADJUSTED: More realistic thresholds
        self.EAR_THRESHOLD = 0.21
        self.MAR_THRESHOLD = 0.7   
        self.HEAD_POSE_THRESHOLD = 45  # FIXED: Increased threshold to reduce false positives
        self.CONSECUTIVE_FRAMES = 25   
        
        # Counters and tracking variables
        self.ear_counter = 0
        self.mar_counter = 0
        self.head_pose_counter = 0
        self.blink_counter = 0
        self.total_blinks = 0
        
        # Temporal tracking
        self.ear_history = deque(maxlen=30)
        self.mar_history = deque(maxlen=30)
        self.head_pose_history = deque(maxlen=30)
        self.blink_history = deque(maxlen=300)  # 10 seconds at 30fps
        
        # FIXED: Head pose calibration - store initial pose as reference
        self.reference_pose = None
        self.pose_calibrated = False
        self.calibration_frames = 0
        self.calibration_poses = []
        
        # Alert system with continuous beeping
        pygame.mixer.init()
        self.alert_sound = None
        self.load_alert_sound()
        self.alert_playing = False
        self.alert_channel = None
        
        # Previous eye state for blink detection
        self.prev_eye_closed = False
        
        # Time tracking
        self.start_time = time.time()
        
    def load_alert_sound(self):
        """Load alert sound with continuous beeping capability"""
        try:
            # Try to load a custom alert sound
            self.alert_sound = pygame.mixer.Sound('alert.wav')
        except:
            # Create a continuous beep sound programmatically
            sample_rate = 22050
            duration = 0.5  # Shorter beep for continuous effect
            frequency = 1000  # Higher frequency for urgency
            t = np.linspace(0, duration, int(sample_rate * duration))
            
            # Create a more urgent beep pattern
            wave = np.sin(frequency * 2 * np.pi * t) * 0.4
            # Add fade in/out to avoid clicking
            fade_samples = int(0.05 * sample_rate)  # 50ms fade
            wave[:fade_samples] *= np.linspace(0, 1, fade_samples)
            wave[-fade_samples:] *= np.linspace(1, 0, fade_samples)
            
            sound_array = (wave * 32767).astype(np.int16)
            sound_array = np.repeat(sound_array.reshape(len(sound_array), 1), 2, axis=1)
            self.alert_sound = pygame.sndarray.make_sound(sound_array)
    
    def get_landmarks(self, landmarks, indices, width, height):
        """Extract specific landmarks and convert to pixel coordinates"""
        points = []
        for idx in indices:
            x = int(landmarks.landmark[idx].x * width)
            y = int(landmarks.landmark[idx].y * height)
            points.append([x, y])
        return np.array(points)
    
    def eye_aspect_ratio(self, eye_landmarks):
        """Enhanced Eye Aspect Ratio (EAR) calculation using multiple points"""
        try:
            # Multiple vertical distances for better accuracy
            A1 = dist.euclidean(eye_landmarks[1], eye_landmarks[7])  # Top to bottom (vertical 1)
            A2 = dist.euclidean(eye_landmarks[2], eye_landmarks[6])  # Top to bottom (vertical 2)
            A3 = dist.euclidean(eye_landmarks[1], eye_landmarks[6])  # Cross vertical
            A4 = dist.euclidean(eye_landmarks[2], eye_landmarks[7])  # Cross vertical
            
            # Horizontal distance (width of the eye)
            C = dist.euclidean(eye_landmarks[0], eye_landmarks[4])  # Outer corner to inner corner
            
            # Enhanced EAR formula using multiple measurements
            if C == 0:  # Prevent division by zero
                return 0.3
            
            ear = (A1 + A2 + A3 + A4) / (4.0 * C)
            return ear
        except:
            return 0.3
    
    def mouth_aspect_ratio(self, mouth_landmarks):
        """Enhanced Mouth Aspect Ratio (MAR) calculation using multiple points"""
        try:
            # Multiple vertical mouth measurements for better accuracy
            A1 = dist.euclidean(mouth_landmarks[1], mouth_landmarks[7])   # Top to bottom (center)
            A2 = dist.euclidean(mouth_landmarks[2], mouth_landmarks[6])   # Top to bottom (left-center)
            A3 = dist.euclidean(mouth_landmarks[3], mouth_landmarks[5])   # Top to bottom (right-center)
            A4 = dist.euclidean(mouth_landmarks[13], mouth_landmarks[19]) # Inner vertical distance
            A5 = dist.euclidean(mouth_landmarks[14], mouth_landmarks[18]) # Inner vertical distance
            
            # Horizontal mouth distance (width)
            C1 = dist.euclidean(mouth_landmarks[0], mouth_landmarks[4])   # Left corner to right corner
            C2 = dist.euclidean(mouth_landmarks[12], mouth_landmarks[16]) # Inner horizontal distance
            
            if C1 == 0 or C2 == 0:  # Prevent division by zero
                return 0.3
                
            # Enhanced MAR formula using multiple measurements
            mar = (A1 + A2 + A3 + A4 + A5) / (3.0 * (C1 + C2))
            return mar
        except:
            return 0.3
    
    def get_head_pose_mediapipe(self, landmarks, width, height):
        """Estimate head pose using MediaPipe 3D landmarks"""
        # Key 3D model points for head pose estimation
        model_points = np.array([
            (0.0, 0.0, 0.0),             # Nose tip
            (0.0, -330.0, -65.0),        # Chin
            (-225.0, 170.0, -135.0),     # Left eye left corner
            (225.0, 170.0, -135.0),      # Right eye right corner  
            (-150.0, -150.0, -125.0),    # Left mouth corner
            (150.0, -150.0, -125.0)      # Right mouth corner
        ])
        
        # Corresponding 2D image points from MediaPipe landmarks
        image_points = np.array([
            [landmarks.landmark[1].x * width, landmarks.landmark[1].y * height],    # Nose tip
            [landmarks.landmark[152].x * width, landmarks.landmark[152].y * height], # Chin
            [landmarks.landmark[33].x * width, landmarks.landmark[33].y * height],   # Left eye corner
            [landmarks.landmark[263].x * width, landmarks.landmark[263].y * height], # Right eye corner
            [landmarks.landmark[61].x * width, landmarks.landmark[61].y * height],   # Left mouth corner
            [landmarks.landmark[291].x * width, landmarks.landmark[291].y * height]  # Right mouth corner
        ], dtype="double")
        
        # Camera internals
        focal_length = width
        center = (width/2, height/2)
        camera_matrix = np.array([
            [focal_length, 0, center[0]],
            [0, focal_length, center[1]],
            [0, 0, 1]
        ], dtype="double")
        
        dist_coeffs = np.zeros((4,1))
        
        # Solve PnP
        try:
            success, rotation_vector, translation_vector = cv2.solvePnP(
                model_points, image_points, camera_matrix, dist_coeffs, 
                flags=cv2.SOLVEPNP_ITERATIVE
            )
            
            if success:
                # Convert rotation vector to angles
                rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
                angles, _, _, _, _, _ = cv2.RQDecomp3x3(rotation_matrix)
                return angles[0], angles[1], angles[2]  # pitch, yaw, roll
            else:
                return 0, 0, 0
        except:
            return 0, 0, 0
    
    def calibrate_head_pose(self, head_angles):
        """FIXED: Calibrate reference head pose for better detection"""
        if not self.pose_calibrated:
            self.calibration_poses.append(head_angles)
            self.calibration_frames += 1
            
            if self.calibration_frames >= 30:  # Calibrate over 30 frames
                # Calculate average pose as reference
                avg_pitch = np.mean([pose[0] for pose in self.calibration_poses])
                avg_yaw = np.mean([pose[1] for pose in self.calibration_poses])
                avg_roll = np.mean([pose[2] for pose in self.calibration_poses])
                
                self.reference_pose = (avg_pitch, avg_yaw, avg_roll)
                self.pose_calibrated = True
                print(f"Head pose calibrated: P:{avg_pitch:.1f} Y:{avg_yaw:.1f} R:{avg_roll:.1f}")
                
        return self.pose_calibrated
    
    def detect_blink(self, ear):
        """Detect blinks based on EAR"""
        eye_closed = ear < self.EAR_THRESHOLD
        
        # Detect transition from closed to open (blink completion)
        if self.prev_eye_closed and not eye_closed:
            self.total_blinks += 1
            self.blink_history.append(time.time())
        
        self.prev_eye_closed = eye_closed
        return eye_closed
    
    def get_blink_frequency(self):
        """Calculate blinks per minute"""
        current_time = time.time()
        # Remove blinks older than 60 seconds
        self.blink_history = deque([t for t in self.blink_history if current_time - t <= 60], 
                                  maxlen=300)
        return len(self.blink_history)
    
    def analyze_drowsiness(self, ear, mar, head_angles, blink_freq):
        """Multi-factor drowsiness analysis"""
        drowsiness_indicators = {
            'eye_closure': False,
            'yawning': False,
            'head_nodding': False,
            'low_blink_rate': False
        }
        
        drowsiness_score = 0
        
        # FIXED: Eye closure detection with better logic
        if ear < self.EAR_THRESHOLD:
            self.ear_counter += 1
            if self.ear_counter >= self.CONSECUTIVE_FRAMES:
                drowsiness_indicators['eye_closure'] = True
                drowsiness_score += 40
        else:
            self.ear_counter = 0
        
        # FIXED: Yawn detection with better threshold
        if mar > self.MAR_THRESHOLD:
            self.mar_counter += 1
            if self.mar_counter >= 15:  # Increased threshold for yawning
                drowsiness_indicators['yawning'] = True
                drowsiness_score += 25
        else:
            self.mar_counter = 0
        
        # FIXED: Head pose analysis with calibration
        if self.pose_calibrated:
            pitch, yaw, roll = head_angles
            ref_pitch, ref_yaw, ref_roll = self.reference_pose
            
            # Calculate deviations from reference pose
            pitch_deviation = abs(pitch - ref_pitch)
            yaw_deviation = abs(yaw - ref_yaw)
            roll_deviation = abs(roll - ref_roll)
            
            # Only trigger if significant deviation from normal pose
            if (pitch_deviation > self.HEAD_POSE_THRESHOLD or 
                yaw_deviation > self.HEAD_POSE_THRESHOLD or 
                roll_deviation > self.HEAD_POSE_THRESHOLD):
                self.head_pose_counter += 1
                if self.head_pose_counter >= 30:  # Increased threshold
                    drowsiness_indicators['head_nodding'] = True
                    drowsiness_score += 20
            else:
                self.head_pose_counter = 0
        
        # Low blink rate detection (normal: 15-20 blinks/minute)
        if blink_freq < 6:  # Very low blink rate
            drowsiness_indicators['low_blink_rate'] = True
            drowsiness_score += 15
        
        # Store history for temporal analysis
        self.ear_history.append(ear)
        self.mar_history.append(mar)
        self.head_pose_history.append(head_angles)
        
        return drowsiness_indicators, drowsiness_score
    
    def trigger_alert(self, drowsiness_score, indicators):
        """Trigger appropriate alerts with continuous beeping for critical states"""
        if drowsiness_score >= 60:  # High drowsiness
            alert_level = "CRITICAL"
            color = (0, 0, 255)  # Red
            should_beep = True
        elif drowsiness_score >= 35:  # Moderate drowsiness
            alert_level = "WARNING"
            color = (0, 165, 255)  # Orange
            should_beep = True
        elif drowsiness_score >= 20:  # Mild drowsiness
            alert_level = "CAUTION"
            color = (0, 255, 255)  # Yellow
            should_beep = False
        else:
            alert_level = "NORMAL"
            color = (0, 255, 0)  # Green
            should_beep = False
        
        # CONTINUOUS BEEPING LOGIC
        if should_beep and self.alert_sound:
            if not self.alert_playing:
                # Start continuous beeping
                try:
                    self.alert_channel = self.alert_sound.play(-1)  # -1 means loop indefinitely
                    self.alert_playing = True
                    print(f"🚨 ALERT STARTED: {alert_level}")
                except:
                    pass
        else:
            # Stop beeping if alert level is low
            if self.alert_playing:
                try:
                    if self.alert_channel:
                        self.alert_channel.stop()
                    self.alert_playing = False
                    print(f"✅ ALERT STOPPED: {alert_level}")
                except:
                    pass
        
        return alert_level, color
    
    def run_detection(self):
        """Main detection loop using MediaPipe"""
        cap = cv2.VideoCapture(0)
        
        # Set camera properties
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        cap.set(cv2.CAP_PROP_FPS, 30)
        
        print("MediaPipe Drowsiness Detection System Started...")
        print("Press 'q' to quit")
        print(f"EAR Threshold: {self.EAR_THRESHOLD}")
        print(f"MAR Threshold: {self.MAR_THRESHOLD}")
        print(f"Consecutive Frames: {self.CONSECUTIVE_FRAMES}")
        print("Calibrating head pose... Please look straight ahead for a few seconds")
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Flip frame horizontally for mirror effect
            frame = cv2.flip(frame, 1)
            height, width, _ = frame.shape
            
            # Convert BGR to RGB for MediaPipe
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            rgb_frame.flags.writeable = False
            
            # Process the frame with MediaPipe
            results = self.face_mesh.process(rgb_frame)
            
            # Convert back to BGR
            rgb_frame.flags.writeable = True
            
            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    # Extract eye landmarks for EAR calculation
                    left_eye = self.get_landmarks(face_landmarks, self.LEFT_EYE_SIMPLE, width, height)
                    right_eye = self.get_landmarks(face_landmarks, self.RIGHT_EYE_SIMPLE, width, height)
                    
                    # Extract mouth landmarks for MAR calculation
                    mouth = self.get_landmarks(face_landmarks, self.MOUTH_SIMPLE, width, height)
                    
                    # Calculate ratios
                    left_ear = self.eye_aspect_ratio(left_eye)
                    right_ear = self.eye_aspect_ratio(right_eye)
                    ear = (left_ear + right_ear) / 2.0
                    
                    mar = self.mouth_aspect_ratio(mouth)
                    
                    # Get head pose
                    head_angles = self.get_head_pose_mediapipe(face_landmarks, width, height)
                    
                    # FIXED: Calibrate head pose first
                    self.calibrate_head_pose(head_angles)
                    
                    # Detect blinks and calculate frequency
                    self.detect_blink(ear)
                    blink_freq = self.get_blink_frequency()
                    
                    # Analyze drowsiness
                    indicators, drowsiness_score = self.analyze_drowsiness(
                        ear, mar, head_angles, blink_freq
                    )
                    
                    # Trigger alerts
                    alert_level, alert_color = self.trigger_alert(drowsiness_score, indicators)
                    
                    # OPTION 1: Use specific eye/mouth regions (current approach)
                    left_eye_full = self.get_landmarks(face_landmarks, self.LEFT_EYE, width, height)
                    right_eye_full = self.get_landmarks(face_landmarks, self.RIGHT_EYE, width, height)
                    mouth_full = self.get_landmarks(face_landmarks, self.MOUTH_SIMPLE, width, height)
                    
                    # OPTION 2: Use ALL 468 landmarks (uncomment below to use all points)
                    # all_landmarks = self.get_landmarks(face_landmarks, self.ALL_FACE_LANDMARKS, width, height)
                    
                    # Draw individual landmark points with different colors and sizes
                    # Left eye points in cyan
                    for point in left_eye_full:
                        cv2.circle(frame, tuple(point), 2, (255, 255, 0), -1)  # Cyan, size 2
                    
                    # Right eye points in cyan
                    for point in right_eye_full:
                        cv2.circle(frame, tuple(point), 2, (255, 255, 0), -1)  # Cyan, size 2
                    
                    # Mouth points in yellow
                    for point in mouth_full:
                        cv2.circle(frame, tuple(point), 2, (0, 255, 255), -1)  # Yellow, size 2
                    
                    # OPTION 2: Draw ALL 468 landmarks (uncomment to use)
                    # for i, point in enumerate(all_landmarks):
                    #     if i < 200:  # First 200 points in red
                    #         cv2.circle(frame, tuple(point), 1, (0, 0, 255), -1)
                    #     elif i < 400:  # Next 200 points in green
                    #         cv2.circle(frame, tuple(point), 1, (0, 255, 0), -1)
                    #     else:  # Remaining points in blue
                    #         cv2.circle(frame, tuple(point), 1, (255, 0, 0), -1)
                    
                    # Draw face bounding rectangle
                    face_points = self.get_landmarks(face_landmarks, self.FACE_OVAL, width, height)
                    x, y, w, h = cv2.boundingRect(face_points)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), alert_color, 2)
                    
                    # Display metrics
                    y_offset = 30
                    cv2.putText(frame, f"Alert Level: {alert_level}", (10, y_offset), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, alert_color, 2)
                    
                    y_offset += 30
                    cv2.putText(frame, f"Drowsiness Score: {drowsiness_score}", (10, y_offset), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                    
                    y_offset += 25
                    cv2.putText(frame, f"EAR: {ear:.3f} (Thresh: {self.EAR_THRESHOLD})", (10, y_offset), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                    
                    y_offset += 20
                    cv2.putText(frame, f"MAR: {mar:.3f} (Thresh: {self.MAR_THRESHOLD})", (10, y_offset), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                    
                    y_offset += 20
                    if self.pose_calibrated:
                        # Show deviation from reference pose
                        ref_pitch, ref_yaw, ref_roll = self.reference_pose
                        pitch_dev = abs(head_angles[0] - ref_pitch)
                        yaw_dev = abs(head_angles[1] - ref_yaw)
                        roll_dev = abs(head_angles[2] - ref_roll)
                        cv2.putText(frame, f"Head Deviation: P:{pitch_dev:.1f} Y:{yaw_dev:.1f} R:{roll_dev:.1f}", 
                                   (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                    else:
                        cv2.putText(frame, f"Calibrating... ({self.calibration_frames}/30)", 
                                   (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
                    
                    y_offset += 20
                    cv2.putText(frame, f"Blinks/min: {blink_freq}", (10, y_offset), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                    
                    y_offset += 20
                    cv2.putText(frame, f"Total Blinks: {self.total_blinks}", (10, y_offset), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                    
                    y_offset += 20
                    cv2.putText(frame, f"EAR Counter: {self.ear_counter}/{self.CONSECUTIVE_FRAMES}", (10, y_offset), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                    
                    # Display active indicators
                    y_offset += 25
                    for indicator, active in indicators.items():
                        if active:
                            cv2.putText(frame, f"{indicator.replace('_', ' ').title()}: ACTIVE", 
                                       (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                            y_offset += 20
            
            # Show frame
            cv2.imshow('MediaPipe Drowsiness Detection System', frame)
            
            # Break loop on 'q' key press
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        # Cleanup
        cap.release()
        cv2.destroyAllWindows()
        
        # Stop any playing alerts
        if self.alert_playing and self.alert_channel:
            try:
                self.alert_channel.stop()
            except:
                pass
                
        self.face_mesh.close()
        pygame.mixer.quit()

In [25]:
if __name__ == "__main__":
    detector = MediaPipeDrowsinessDetector()
    detector.run_detection()

MediaPipe Drowsiness Detection System Started...
Press 'q' to quit
EAR Threshold: 0.21
MAR Threshold: 0.7
Consecutive Frames: 25
Calibrating head pose... Please look straight ahead for a few seconds
🚨 ALERT STARTED: WARNING
Head pose calibrated: P:-176.4 Y:2.6 R:-1.8
✅ ALERT STOPPED: CAUTION
🚨 ALERT STARTED: WARNING
✅ ALERT STOPPED: CAUTION
🚨 ALERT STARTED: CRITICAL
✅ ALERT STOPPED: CAUTION
🚨 ALERT STARTED: CRITICAL
✅ ALERT STOPPED: CAUTION
🚨 ALERT STARTED: CRITICAL
✅ ALERT STOPPED: CAUTION
🚨 ALERT STARTED: CRITICAL
✅ ALERT STOPPED: CAUTION
🚨 ALERT STARTED: CRITICAL
✅ ALERT STOPPED: CAUTION
🚨 ALERT STARTED: CRITICAL
✅ ALERT STOPPED: CAUTION
🚨 ALERT STARTED: WARNING
✅ ALERT STOPPED: CAUTION
🚨 ALERT STARTED: WARNING
✅ ALERT STOPPED: CAUTION
🚨 ALERT STARTED: WARNING
✅ ALERT STOPPED: CAUTION
